# 5. Find largest observations (R)

In [1]:
library(tidyverse)
library(feather)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## Load data

In [2]:
pipelines_2010 <- read_feather('../preprocessed_data/pipelines_2010_selected_2019-08-03.feather')
glimpse(pipelines_2010)

Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”

Observations: 5,294
Variables: 5
$ OPERATOR_ID <dbl> 300, 300, 300, 395, 402, 473, 473, 515, 879, 994, 994, 99…
$ YEAR        <dbl> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 201…
$ NAME        <chr> "PLAINS PIPELINE, L.P.", "PLAINS PIPELINE, L.P.", "PLAINS…
$ COMMODITY   <chr> "Crude Oil", "HVL", "Refined and/or Petroleum Product (no…
$ MILES       <dbl> 2101.060, 72.900, 0.000, 662.300, 1.550, 12.000, 0.000, 3…


## 5.1 Find largest observations

In [3]:
pipelines_2010 %>%
    group_by(OPERATOR_ID) %>%
    top_n(1, MILES) %>%
    ungroup() %>%
    top_n(10, MILES) %>%
    arrange(desc(MILES))

OPERATOR_ID,YEAR,NAME,COMMODITY,MILES
<dbl>,<dbl>,<chr>,<chr>,<dbl>
31618,2018,ENTERPRISE PRODUCTS OPERATING LLC,HVL,7726.183
32109,2018,"ONEOK NGL PIPELINE, LLC",HVL,4531.480
2552,2015,COLONIAL PIPELINE CO,Refined and/or Petroleum Product (non-HVL),4500.920
1845,2015,"BUCKEYE PARTNERS, LP",Refined and/or Petroleum Product (non-HVL),4266.430
22610,2018,"MAGELLAN PIPELINE COMPANY, LP",Refined and/or Petroleum Product (non-HVL),3926.400
15674,2016,PLANTATION PIPE LINE CO,Refined and/or Petroleum Product (non-HVL),2564.730
300,2016,"PLAINS PIPELINE, L.P.",Crude Oil,2312.330
11169,2017,"ENBRIDGE ENERGY, LIMITED PARTNERSHIP",Crude Oil,2290.000
31684,2010,CONOCOPHILLIPS,Refined and/or Petroleum Product (non-HVL),2283.790


#### Largest for the sum of all types

In [4]:
unique(pipelines_2010$COMMODITY)

[1] "Crude Oil"                                 
[2] "HVL"                                       
[3] "Refined and/or Petroleum Product (non-HVL)"
[4] "CO2"                                       
[5] "Fuel Grade Ethanol (dedicated system)"

Check whether any operator ID has more than the maximum of five expected entries.

In [5]:
pipelines_2010 %>%
    group_by(YEAR, OPERATOR_ID) %>%
    summarize(n = n()) %>%
    ungroup() %>%
    {table(.$n)}


   1    2    3    4 
3307  565  251   26 

Everything seems to be A-Okay.

In [6]:
pipelines_2010 %>%
    group_by(YEAR, OPERATOR_ID) %>%
    summarize(MILES = sum(MILES), NAME = last(NAME)) %>%
    ungroup() %>%
    group_by(OPERATOR_ID) %>%
    top_n(1, MILES) %>%
    ungroup() %>%
    top_n(10, MILES) %>%
    arrange(desc(MILES))

YEAR,OPERATOR_ID,MILES,NAME
<dbl>,<dbl>,<dbl>,<chr>
2018,31618,8325.499,ENTERPRISE PRODUCTS OPERATING LLC
2018,32109,4756.610,"ONEOK NGL PIPELINE, LLC"
2014,22610,4505.500,"MAGELLAN PIPELINE COMPANY, LP"
2015,2552,4500.920,COLONIAL PIPELINE CO
2017,31684,4474.100,PHILLIPS 66 PIPELINE LLC
2015,1845,4298.290,"BUCKEYE PARTNERS, LP"
2018,32147,4037.200,MARATHON PIPE LINE LLC
2010,18718,3466.740,SUNOCO PIPELINE L.P.
2013,4906,2944.400,EXXONMOBIL PIPELINE CO
